<a href="https://colab.research.google.com/github/josmyrose/digital_clinical_ml/blob/main/01_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================
# 📘 01_data_preparation.ipynb
# Author: Josmy Mathew
# Project: AI-Digital-Twin-for-Clinical-Data
# Description:
#   Load, clean, and integrate Synthea EHR data into
#   structured longitudinal patient time-series tables.
# ==============================================================



**Step 1 — Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
# Display settings
pd.set_option('display.max_columns', None)


**Step 2 — Load Synthea CSV Data**

In [ ]:
data_dir = Path("/content/drive/MyDrive/synthea_sample_data_csv_latest")
files = {f.stem: f for f in data_dir.glob("*.csv")}

dfs = {}
for name, path in files.items():
    dfs[name] = pd.read_csv(path)
    print(f"Loaded {name}: {dfs[name].shape}")

Loaded supplies: (3149, 6)
Loaded observations: (100980, 9)
Loaded claims_transactions: (104272, 33)
Loaded medications: (5701, 13)
Loaded devices: (640, 7)
Loaded claims: (12115, 31)
Loaded careplans: (337, 9)
Loaded payers: (10, 22)
Loaded organizations: (273, 11)
Loaded immunizations: (1535, 6)
Loaded allergies: (85, 15)
Loaded providers: (273, 13)
Loaded payer_transitions: (4097, 8)
Loaded imaging_studies: (29193, 13)
Loaded conditions: (4294, 7)
Loaded encounters: (6414, 15)
Loaded patients: (109, 28)
Loaded procedures: (19287, 10)


In [ ]:
# Parse datetimes for columns that look like dates
def parse_dates(df):
    for c in df.columns:
        if any(k in c.lower() for k in ['date','start','stop','birth']):
            try:
                df[c] = pd.to_datetime(df[c], errors='coerce')
            except Exception:
                pass
    return df

for k in list(dfs.keys()):
    dfs[k] = parse_dates(dfs[k])

# quick shapes and sample columns
for k, df in dfs.items():
    print(f"{k}: shape={df.shape}")
    print(df.columns.tolist())
    print()


supplies: shape=(3149, 6)
['DATE', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'QUANTITY']

observations: shape=(100980, 9)
['DATE', 'PATIENT', 'ENCOUNTER', 'CATEGORY', 'CODE', 'DESCRIPTION', 'VALUE', 'UNITS', 'TYPE']

claims_transactions: shape=(104272, 33)
['ID', 'CLAIMID', 'CHARGEID', 'PATIENTID', 'TYPE', 'AMOUNT', 'METHOD', 'FROMDATE', 'TODATE', 'PLACEOFSERVICE', 'PROCEDURECODE', 'MODIFIER1', 'MODIFIER2', 'DIAGNOSISREF1', 'DIAGNOSISREF2', 'DIAGNOSISREF3', 'DIAGNOSISREF4', 'UNITS', 'DEPARTMENTID', 'NOTES', 'UNITAMOUNT', 'TRANSFEROUTID', 'TRANSFERTYPE', 'PAYMENTS', 'ADJUSTMENTS', 'TRANSFERS', 'OUTSTANDING', 'APPOINTMENTID', 'LINENOTE', 'PATIENTINSURANCEID', 'FEESCHEDULEID', 'PROVIDERID', 'SUPERVISINGPROVIDERID']

medications: shape=(5701, 13)
['START', 'STOP', 'PATIENT', 'PAYER', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION']

devices: shape=(640, 7)
['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', '

/tmp/ipython-input-2306563396.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors='coerce')


In [ ]:
# Parse datetimes for columns that look like dates
def parse_dates(df):
    for c in df.columns:
        if any(k in c.lower() for k in ['date','start','stop','birth']):
            try:
                df[c] = pd.to_datetime(df[c], errors='coerce')
            except Exception:
                pass
    return df

for k in list(dfs.keys()):
    dfs[k] = parse_dates(dfs[k])

# quick shapes and sample columns
for k, df in dfs.items():
    print(f"{k}: shape={df.shape}")
    print(df.columns.tolist())
    print()


supplies: shape=(3149, 6)
['DATE', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'QUANTITY']

observations: shape=(100980, 9)
['DATE', 'PATIENT', 'ENCOUNTER', 'CATEGORY', 'CODE', 'DESCRIPTION', 'VALUE', 'UNITS', 'TYPE']

claims_transactions: shape=(104272, 33)
['ID', 'CLAIMID', 'CHARGEID', 'PATIENTID', 'TYPE', 'AMOUNT', 'METHOD', 'FROMDATE', 'TODATE', 'PLACEOFSERVICE', 'PROCEDURECODE', 'MODIFIER1', 'MODIFIER2', 'DIAGNOSISREF1', 'DIAGNOSISREF2', 'DIAGNOSISREF3', 'DIAGNOSISREF4', 'UNITS', 'DEPARTMENTID', 'NOTES', 'UNITAMOUNT', 'TRANSFEROUTID', 'TRANSFERTYPE', 'PAYMENTS', 'ADJUSTMENTS', 'TRANSFERS', 'OUTSTANDING', 'APPOINTMENTID', 'LINENOTE', 'PATIENTINSURANCEID', 'FEESCHEDULEID', 'PROVIDERID', 'SUPERVISINGPROVIDERID']

medications: shape=(5701, 13)
['START', 'STOP', 'PATIENT', 'PAYER', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION']

devices: shape=(640, 7)
['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', '

In [ ]:
# Save all loaded raw tables as processed (sanitized copies) to keep versioned intermediate files
PROCESSED = Path("/content/drive/MyDrive/synthea_sample_data_csv_latest/Processed")
PROCESSED.mkdir(exist_ok=True)

print("Saved processed CSVs to:", PROCESSED)
# Drop PII from patients
drop_cols = ['SSN','DRIVERS','PASSPORT','ADDRESS','PHONE']
patients = dfs.get('patients').copy()
for c in drop_cols:
    if c in patients.columns:
        patients.drop(columns=[c], inplace=True)

# Save sanitized patients and a small sample of the dataset for the repo
patients.to_csv(PROCESSED/'patients_sanitized.csv', index=False)
print("Saved patients_sanitized.csv")


Saved processed CSVs to: /content/drive/MyDrive/synthea_sample_data_csv_latest/Processed
Saved patients_sanitized.csv


In [ ]:
# Save all loaded raw tables as processed (sanitized copies) to keep versioned intermediate files
for name, df in dfs.items():
    # remove obvious PII columns if present (simple list)
    df2 = df.copy()
    for pi in ['SSN','PASSPORT','DRIVERS','ADDRESS','PHONE','LAT','LON']:
        if pi in df2.columns:
            df2.drop(columns=[pi], inplace=True)
    df2.to_csv(PROCESSED/f'{name}_processed.csv', index=False)
print("Saved processed CSVs to:", PROCESSED)


Saved processed CSVs to: /content/drive/MyDrive/synthea_sample_data_csv_latest/Processed


# **Quick Summary Matrix**

In [ ]:
# Basic summary
print("✅ Data loaded:")
for name in ['patients','encounters','observations','conditions']:
    if name in dfs:
        print(f"{name.capitalize()}: {dfs[name].shape}")


✅ Data loaded:
Patients: (109, 28)
Encounters: (6414, 15)
Observations: (100980, 9)
Conditions: (4294, 7)


# REFERENCES



1.   https://synthea.mitre.org/downloads
2.   List item

